In [1]:
import os
import sys
import re
import pandas as pd
#import yabul

In [2]:
def read_fasta(filename):
    """
    Parse a fasta file to a pandas DataFrame.
    Compression is supported (via pandas read_csv) and is inferred by
    extension: '.gz', '.bz2', '.zip', or '.xz'.
    Parameters
    ----------
    filename : string
    Returns
    -------
    pandas.DataFrame with columns "description" and "sequence". The index of the
    DataFrame is the "sequence ID", i.e. the first space-separated token of the
    description.
    """
    # We (mis-) use pandas to read the file.
    lines = pd.read_csv(
        filename,
        header=None,
        skip_blank_lines=True,
        dtype=str,
        na_filter=False,
        quoting=3,  # QUOTE_NONE
        encoding = "ISO-8859-1",
        sep="\0",  # null separator: never split, always read one column
    )
    assert lines.shape[1] == 1  # one column
    lines.columns = ["raw"]

    result = []
    current_id = None
    pieces = []
    for line in lines.raw:
        if line.startswith(">"):
            if current_id is not None:
                result.append((current_id, "".join(pieces)))
                pieces.clear()
            current_id = line[1:]
        else:
            pieces.append(line)

    # Handle last sequence
    if current_id is not None:
        result.append((current_id, "".join(pieces)))

    result = pd.DataFrame(result, columns=["description", "sequence"])
    result.index = result.description.str.split().str.get(0)
    result.index.name = "id"

    return result

In [18]:
# download protein and DNA spike sequences from GISAID

# read fasta files, including the reference

#seq_file='./fa/spikenuc0427.fasta'
seq_file='./fa/spikeprot0423/spikeprot0423.fasta'
#ref_file='spikeprot_ref_alpha.fasta'

spike_df = read_fasta(seq_file).rename(columns={"description": "sequence_id"})
#spike_ref =yabul.read_fasta(ref_file).rename(columns={"description": "sequence_id"})
spike_df.reset_index(inplace=True, drop=True)
#spike_ref.reset_index(inplace=True, drop=True)
spike_df.head()

,sequence_id,sequence
0,Spike|hCoV-19/Wuhan/WIV04/2019|2019-12-30|EPI_...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,Spike|hCoV-19/Philippines/PH-PGC-03696/2020|20...,MFVFLVLLPLVFSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,Spike|hCoV-19/Belgium/UZA-UA-8350/2021|2021-01...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,Spike|hCoV-19/Belgium/UZA-UA-CV0614875118/2021...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,Spike|hCoV-19/Belgium/UZA-UA-2735/2021|2021-01...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [19]:
spike_split = spike_df.sequence_id.str.split('|')
spike_df['isolate']=[el[1] for el in spike_split]
spike_df['date']=[el[2] for el in spike_split]

In [20]:
spike_df.isolate=spike_df.isolate.apply(lambda x: re.sub('^hcov-19/','', x, flags=re.IGNORECASE))
spike_df.isolate=spike_df.isolate.apply(lambda x: re.sub(' ','_', x, flags=re.IGNORECASE))
spike_df['date']=pd.to_datetime(spike_df.date, errors='coerce')

In [6]:
! wget https://raw.githubusercontent.com/cov-lineages/pango-designation/master/lineages.csv -O fa/lineages.csv
! wget https://raw.githubusercontent.com/cov-lineages/pango-designation/master/pango_designation/alias_key.json -O fa/alias_key.json
lineages=pd.read_csv('fa/lineages.csv')

--2023-04-28 13:18:04--  https://raw.githubusercontent.com/cov-lineages/pango-designation/master/lineages.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81006129 (77M) [text/plain]
Saving to: ‘fa/lineages.csv’

fa/lineages.csv     100%[===================>]  77.25M   110MB/s    in 0.7s    

2023-04-28 13:18:06 (110 MB/s) - ‘fa/lineages.csv’ saved [81006129/81006129]

--2023-04-28 13:18:07--  https://raw.githubusercontent.com/cov-lineages/pango-designation/master/pango_designation/alias_key.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response..

In [21]:
lineages=pd.read_csv('fa/lineages.csv')

In [22]:
lineages[lineages['lineage']=='XBB.1.16']

,taxon,lineage
2230482,Germany/BY-RKI-I-1104685/2023,XBB.1.16
2230483,USA/NY-CDC-814743580/2023,XBB.1.16
2230709,Denmark/DCGC-642389/2023,XBB.1.16
2230710,Singapore/R16MR13/2023,XBB.1.16
2230711,Singapore/R16MR11/2023,XBB.1.16
2230712,Singapore/R16MR9/2023,XBB.1.16
2230713,Italy/LAZ-FPG-1095/2023,XBB.1.16
2230714,Singapore/R15MR15/2023,XBB.1.16
2230715,Singapore/R15MR11/2023,XBB.1.16
2230716,USA/WA-GBW-H20-268-5306/2023,XBB.1.16


In [23]:
lineages_new = lineages.merge(spike_df, how='inner', left_on='taxon', right_on='isolate')

In [10]:
# DNA
lineages_new=lineages_new[~lineages_new.sequence.apply(lambda x: bool(re.search('[^ATCG]', x, flags=re.IGNORECASE)))]

In [24]:
# protein
lineages_new=lineages_new[~lineages_new.sequence.apply(lambda x: bool(re.search('[XBOUZJ]', x, flags=re.IGNORECASE)))]

In [25]:
lineages_new[lineages_new.lineage=='XBB.1.5']['sequence'].iloc[0]

'MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPALPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLDVYQKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKEGNFKNLREFVFKNIDGYFKIYSKHTPINLERDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPVDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFHEVFNATTFASVYAWNRKRISNCVADYSVIYNFAPFFAFKCYGVSPTKLNDLCFTNVYADSFVIRGNEVSQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNKLDSKPSGNYNYLYRLFRKSKLKPFERDISTEIYQAGNKPCNGVAGPNCYSPLQSYGFRPTYGVGHQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEYVNNSYECDIPIGAGICASYQTQTKSHRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLKRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKYFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNHNAQALNTLVKQLSSKFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQS

In [26]:
lineages_new

,taxon,lineage,sequence_id,sequence,isolate,date
0,Belgium/UZA-UA-48355442/2023,XBZ,Spike|hCoV-19/Belgium/UZA-UA-48355442/2023|202...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,Belgium/UZA-UA-48355442/2023,2023-01-05
7,Germany/HE-RKI-I-1084179/2022,XBZ,Spike|hCoV-19/Germany/HE-RKI-I-1084179/2022|20...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,Germany/HE-RKI-I-1084179/2022,2022-12-23
13,Germany/HE-RKI-I-1085588/2022,XBZ,Spike|hCoV-19/Germany/HE-RKI-I-1085588/2022|20...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,Germany/HE-RKI-I-1085588/2022,2022-12-28
14,Germany/NW-RKI-I-1088080/2023,XBZ,Spike|hCoV-19/Germany/NW-RKI-I-1088080/2023|20...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,Germany/NW-RKI-I-1088080/2023,2023-01-09
16,USA/CA-HLX-STM-2K7QE9FQ2/2022,XBZ,Spike|hCoV-19/USA/CA-HLX-STM-2K7QE9FQ2/2022|20...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,USA/CA-HLX-STM-2K7QE9FQ2/2022,2022-12-19
...,...,...,...,...,...,...
2355665,England/BRBR-32716E0D/2023,BQ.1.2.2,Spike|hCoV-19/England/BRBR-32716E0D/2023|2023-...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,England/BRBR-32716E0D/2023,2023-03-29
2355667,England/BRBR-32716E67/2023,BQ.1.2.2,Spike|hCoV-19/England/BRBR-32716E67/2023|2023-...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,England/BRBR-32716E67/2023,2023-03-29
2355668,England/BRBR-32717A35/2023,BQ.1.2.2,Spike|hCoV-19/England/BRBR-32717A35/2023|2023-...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,England/BRBR-32717A35/2023,2023-03-29
2355670,England/BRBR-32717A44/2023,BQ.1.2.2,Spike|hCoV-19/England/BRBR-32717A44/2023|2023-...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,England/BRBR-32717A44/2023,2023-03-29


In [13]:
lineages_new.to_csv('COVID/data/lineage_nuc.csv', index=False)

In [27]:
lineages_new.to_csv('COVID/data/lineage_prot.csv', index=False)

In [28]:
lineage_max_date = lineages_new.groupby("lineage", as_index=False)["date"].max()
lineage_min_date = lineages_new.groupby("lineage", as_index=False)["date"].min()
lineage_min_date = lineage_min_date.merge(lineages_new, how='left')
lineage_max_date = lineage_max_date.merge(lineages_new, how='left')
lineage_max_date

,lineage,date,taxon,sequence_id,sequence,isolate
0,A,2021-02-02,Netherlands/UT-RIVM-11945/2021,Spike|hCoV-19/Netherlands/UT-RIVM-11945/2021|2...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Netherlands/UT-RIVM-11945/2021
1,A.1,2020-07-22,Ecuador/USFQ-165/2020,Spike|hCoV-19/Ecuador/USFQ-165/2020|2020-07-22...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Ecuador/USFQ-165/2020
2,A.11,2020-04-19,England/LOND-D604F/2020,Spike|hCoV-19/England/LOND-D604F/2020|2020-04-...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,England/LOND-D604F/2020
3,A.12,2020-12-11,England/CAMC-C91F38/2020,Spike|hCoV-19/England/CAMC-C91F38/2020|2020-12...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,England/CAMC-C91F38/2020
4,A.15,2020-08-10,Denmark/DCGC-2446/2020,Spike|hCoV-19/Denmark/DCGC-2446/2020|2020-08-1...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Denmark/DCGC-2446/2020
...,...,...,...,...,...,...
4833,XZ,2022-05-12,USA/NY-PRL-220513_00E16/2022,Spike|hCoV-19/USA/NY-PRL-220513_00E16/2022|202...,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...,USA/NY-PRL-220513_00E16/2022
4834,Y.1,2021-01-17,Portugal/PT2850/2021,Spike|hCoV-19/Portugal/PT2850/2021|2021-01-17|...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Portugal/PT2850/2021
4835,Y.1,2021-01-17,Switzerland/GE-32997519/2021,Spike|hCoV-19/Switzerland/GE-32997519/2021|202...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Switzerland/GE-32997519/2021
4836,Y.1,2021-01-17,Portugal/PT2458/2021,Spike|hCoV-19/Portugal/PT2458/2021|2021-01-17|...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Portugal/PT2458/2021


In [ ]:
#lineage_min_date=lineage_min_date.rename(columns={'date': 'min_date'})
#lineage_max_date=lineage_max_date.rename(columns={'date': 'max_date'})

In [29]:
lineage_min_date.to_csv('COVID/data/lineage_min_date_prot.csv', index=False)
lineage_max_date.to_csv('COVID/data/lineage_max_date_prot.csv', index=False)

In [15]:
lineage_min_date.to_csv('COVID/data/lineage_min_date_nuc.csv', index=False)
lineage_max_date.to_csv('COVID/data/lineage_max_date_nuc.csv', index=False)

In [30]:
lineage_freq=lineages_new.groupby(["lineage","sequence"], as_index=False).size()
lineage_freq_max=lineage_freq.groupby("lineage", as_index=False)['size'].max()
lineage_freq_max=lineage_freq_max.merge(lineage_freq)
lineage_freq_max

,lineage,size,sequence
0,A,1217,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,A.1,2037,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,A.11,4,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,A.12,3,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,A.15,47,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
...,...,...,...
2868,XW,33,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
2869,XY,24,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
2870,XZ,48,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
2871,Y.1,21,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [31]:
# protein
lineage_freq_max.to_csv('COVID/data/lineage_freq_max_prot.csv', index=False)

In [17]:
# DNA
lineage_freq_max.to_csv('COVID/data/lineage_freq_max_nuc.csv', index=False)